In [2]:
#load datasets, library, etc
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
import pip
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
pd.set_option('display.max_colwidth', -1)

import string
import re
from urlextract import URLExtract
from collections import Counter

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB


ModuleNotFoundError: No module named 'Sastrawi'

In [ ]:
# function collection
def replace_multiple(mainString, toBeReplaces, newString):
    for elem in toBeReplaces :
        if elem in mainString :
            mainString = mainString.replace(elem, newString)
    return  mainString

def get_emails(str):
    regex = r'([\w0-9._-]+@[\w0-9._-]+\.[\w0-9_-]+)'
    #(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])
    return re.findall(regex, str, re.M|re.I)

def get_arr_most_occured_word_using_counter(ds):
    list_spam_msg = ds.loc[ds['class'] == 0].clean_msg
    string_spam_msg = list_spam_msg.str.cat(sep=', ')
    string_spam_msg_splitted = string_spam_msg.split()
    counter = Counter(string_spam_msg_splitted)
    return counter

def vektor(ds, feature):
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')
    cv = CountVectorizer(lowercase=True,ngram_range=(1,1),tokenizer=token.tokenize, vocabulary=feature)
    # CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, 
    #                 lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, 
    #                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, 
    #                 max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, 
    #                 dtype=<class ‘numpy.int64’>)[source]
    text_counts=cv.fit_transform(ds['clean_msg'])
    return cv, text_counts

# select datasets and reformat
def select_dataset(ds):
    if(ds == 1):
        df = pd.read_csv('datasets\singlish.txt', sep="\t", header=None, names = ["class", "msg"])
        df = df[['msg', 'class']] # reorder index
        df['class'] = df['class'].replace(['ham'], 0).replace(['spam'], 1) # represents class using number
        df.language = 'english'
        df.stop_words = nltk.corpus.stopwords.words(df.language)
    elif(ds == 2):
        df = pd.read_csv('datasets\indo.csv')
        df.columns = ['msg', 'class'] #rename
        df['class'] = df['class'].replace([1,2], 1) # promo & fraud = spam (1)
        df.language = 'indonesian'
        df.stop_words = nltk.corpus.stopwords.words(df.language)
    else:
        return false
    return df

models = {
"knn": KNeighborsClassifier(n_neighbors=1),
"naive_bayes": GaussianNB(),
"decision_tree": DecisionTreeClassifier(),
"random_forest": RandomForestClassifier(n_estimators=100),
"multi_nb": MultinomialNB(),
"bernoulli_nb":BernoulliNB(),
}

In [ ]:
# preprocessing text
extractor = URLExtract()
lem = nltk.WordNetLemmatizer()
porter = nltk.PorterStemmer()
indo_factory_stemmer = StemmerFactory()
indo_factory_stopwords = StopWordRemoverFactory()
indo_stemmer = indo_factory_stemmer.create_stemmer()
indo_stopwords = indo_factory_stopwords.create_stop_word_remover()

def preproccessing_data(ds):
    processed_msg = []
    for msg in ds['msg']:
        clean_msg = msg.lower()
        # tokenization
        # spelling correction
        # masking
        clean_msg = replace_multiple(clean_msg, get_emails(clean_msg), ' EMAILADDRESS ')
        clean_msg = replace_multiple(clean_msg, extractor.find_urls(msg), ' WEBADDRESS ')
        clean_msg = re.sub(r"(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:']))", ' WEBADDRESS ',clean_msg) # ^ not quite enough

        clean_msg = re.sub(r"(?:\*\d+)+#", ' USSDCODE ',clean_msg)
        clean_msg = re.sub(r"\s+(?=\d{2}(?:\d{2})?-\d{1,2}-\d{1,2})", ' DATETIME ',clean_msg)
        clean_msg = re.sub(r"(?:[rp\£\$\€]{1}[,\d]+.?\d*)",' PRICEOFFER ',clean_msg)
        clean_msg = re.sub(r"(rp|USD|EUR|€|\$)\s?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))|(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)\s?(USD|EUR|€|\$|Rp|rb|ribu)",' PRICEOFFER ',clean_msg)
        clean_msg = re.sub(r"(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?", ' PHONENUMBER ', clean_msg)
        clean_msg = re.sub(r"\d+(\.\d+)?", ' SOMENUMBER ',clean_msg)
        # striping symbols
        clean_msg = clean_msg.translate(str.maketrans(" "," ", string.punctuation))
        # from ppt [not implemented (yet), here just in case]
    #     clean_msg = re.sub(r"\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b", 'PHONENUMBER',clean_msg
    #     clean_msg = re.sub(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", 'EMAILADDRESS',clean_msg)
    #     clean_msg = re.sub(r'[^\w]', ' ', clean_msg) # white space? dunno for what
        if(ds.language == 'english'):
            # lemmatizer
            clean_msg = ''.join([lem.lemmatize(word) for word in clean_msg]) # not really works
            # stemming
            clean_msg = ''.join([porter.stem(word) for word in clean_msg])
            # stop words
            stop_words = set(stopwords.words(ds.language)) 
            found_stop_words = [w for w in word_tokenize(clean_msg) if w in stop_words] 
            clean_msg = replace_multiple(clean_msg, found_stop_words, '')
        else: # indo
            # stemming
            clean_msg   = indo_stemmer.stem(clean_msg)
            # stop words
            stop_words = set(stopwords.words(ds.language))
            clean_msg = indo_stopwords.remove(clean_msg) # sastrawi
        processed_msg.append(clean_msg)
    ds['clean_msg'] = processed_msg
    return ds

In [ ]:
def get_dict_most_occured_word_manual(ds):
    msg_string =' '.join(ds.clean_msg) # all clean msg row into one string
    msg_string_list = msg_string.lower().split() # split string into array
    unique_words = set(msg_string_list)
    dictwords = dict()
    for words in unique_words:
        dictwords[words] = [msg_string_list.count(words)]
    real_msg = ds[ds['class'] == 0]
    spam_msg = ds[ds['class'] == 1]
    string_real_msg = ' '.join(real_msg.clean_msg)
    string_spam_msg = ' '.join(spam_msg.clean_msg)
    list_real_msg = string_real_msg.lower().split()
    list_spam_msg = string_spam_msg.lower().split()
    for words in unique_words :
        count_real_msg =list_real_msg.count(words)
        count_spam_msg =list_spam_msg.count(words)
        dictwords[words].append(count_real_msg)
        dictwords[words].append(count_spam_msg)
        dictwords[words].append(count_spam_msg-count_real_msg)
    sortfreq = sorted(dictwords.items(), key=lambda x: x[1][3], reverse=True)
    dictfreq = sortfreq[:100]
    return dictfreq
# [count of word occured (all), count of word occured (real), count of word occured (spam), spam {minus} real]

In [ ]:
# singlish datasets
df = select_dataset(1)
df = preproccessing_data(df)
mo st_occured_word_counter = get_arr_most_occured_word_using_counter(df)
most_occured_word_dictionary = get_dict_most_occured_word_manual(df)
# df['clean_msg'].head(10)
# export_excel = df.to_excel (r'export_dataframe.xlsx', index = None, header=True)

In [ ]:
# indo datasets
dfX = select_dataset(2)
dfX = preproccessing_data(dfX)
most_occured_word_counterX = get_arr_most_occured_word_using_counter(dfX)
most_occured_word_dictionaryX = get_dict_most_occured_word_manual(dfX)

# df['clean_msg'].head(10)
# export_excel = df.to_excel (r'export_dataframe.xlsx', index = None, header=True)

In [ ]:
%%time
# feature = ['somenumber','phonenumber','priceoffer','free','txt','cll','webaddress','mobile','prize','clim','urgent','service','reply','text','claim','win']
dictionary = most_occured_word_dictionary[:80]
feature = [i[0] for i in dictionary]
cv, text_counts = vektor(df, feature)
x_train, x_test, y_train, y_test = train_test_split(text_counts, df['class'], test_size=0.2, random_state=1)
clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)
print('Multinomial NB report summary datasets singlish : ')
print(classification_report(y_test, predicted,target_names=['not spam','spam']))

In [ ]:
%%time
dictionaryX = most_occured_word_dictionaryX[:30]
featureX = [i[0] for i in dictionaryX]
additional = ['rek','tukar','toyota','juta','raih','yaris','bpk','extra','operator','mendptkan','nama']
featureX = featureX + additional
cvX, text_countsX = vektor(dfX, featureX)
X_train, X_test, Y_train, Y_test = train_test_split(text_countsX, dfX['class'], test_size=0.2, random_state=1)
clfX = MultinomialNB().fit(X_train, Y_train)
predictedX = clfX.predict(X_test)
print('Multinomial NB report summary datasets indo : ')
print(classification_report(Y_test, predictedX,target_names=['not spam','spam']))

In [ ]:
%%time
model_name="knn"
model = models[model_name]
cvX, text_countsX = vektor(dfX, featureX)
X_train, X_test, Y_train, Y_test = train_test_split(text_countsX, dfX['class'], test_size=0.2, random_state=1)
clfX = model.fit(X_train, Y_train)
predictedX = clfX.predict(X_test)
print(model_name, 'report summary datasets indo : ')
print(classification_report(Y_test, predictedX,target_names=['not spam','spam']))

In [ ]:
%%time
model_name="decision_tree"
model = models[model_name]
cvX, text_countsX = vektor(dfX, featureX)
X_train, X_test, Y_train, Y_test = train_test_split(text_countsX, dfX['class'], test_size=0.2, random_state=1)
clfX = model.fit(X_train, Y_train)
predictedX = clfX.predict(X_test)
print(model_name, 'report summary datasets indo : ')
print(classification_report(Y_test, predictedX,target_names=['not spam','spam']))

In [ ]:
%%time
model_name="random_forest"
model = models[model_name]
cvX, text_countsX = vektor(dfX, featureX)
X_train, X_test, Y_train, Y_test = train_test_split(text_countsX, dfX['class'], test_size=0.2, random_state=1)
clfX = model.fit(X_train, Y_train)
predictedX = clfX.predict(X_test)
print(model_name, 'report summary datasets indo : ')
print(classification_report(Y_test, predictedX,target_names=['not spam','spam']))

In [ ]:
%%time
model_name="bernoulli_nb"
model = models[model_name]
cvX, text_countsX = vektor(dfX, featureX)
X_train, X_test, Y_train, Y_test = train_test_split(text_countsX, dfX['class'], test_size=0.2, random_state=1)
clfX = model.fit(X_train, Y_train)
predictedX = clfX.predict(X_test)
print(model_name, 'report summary datasets indo : ')
print(classification_report(Y_test, predictedX,target_names=['not spam','spam']))